In [9]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)
import pandas as pd
import logging
import os
import csv
from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)



In [14]:
folder_name = 'chenyu_51657'

class data:
    def __init__(self, file_name) -> None:

        self.file_name = file_name.replace('.csv','')
        self.file_path = folder_name+'/'+self.file_name+'.csv'
        self.type = self.file_name.split('_')[0]

        if self.type == 'label':

            self.p = float(self.file_name.split('_')[1])
            self.len = int(self.file_name.split('_')[2])

            with open(self.file_path, mode = 'r') as f:
                self.data = pd.DataFrame([[i[0],list(i[1])] for i in csv.reader(f)])
                self.data.columns = ['text', 'labels']
            
        elif self.type == 'mask':

            self.p = 1.0
            self.len = int(self.file_name.split('_')[1])

            with open(self.file_path, mode = 'r') as f:
                self.data = pd.DataFrame([['replace MASK', i[0], i[1]] for i in csv.reader(f)])
                self.data.columns = ["prefix", "input_text", "target_text"]
            
        self.train, self.test = train_test_split(self.data, test_size=0.2, random_state=96)
        del self.data

label_data = data('label_0.2_16.csv')
print(label_data.test)

                   text                                            labels
13571  最近我读了一则新闻，岁少年涂振鹏  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
24382  文化日新月异，各种各样的知识浩如  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
1613   到了具体地品评文章作品，有的话和  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
38180  鸟们有时在天空有时停在树枝上婉转  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
35766  别说我一毛不拔，今天花上一毛钱，  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
...                 ...                                               ...
25666  做事要不空运控制自己，适可而止才  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
10700  张唐就是因为最后的两位皇帝穷奢极  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21161  蒙住自己的眼睛，并不等于世界蒙住  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
13684  但如今双方对垒已得不发不是你死就  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
34363     在李太太面前，李先生一向。  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

[8165 rows x 2 columns]


In [ ]:
# Optional model configuration
model_args = MultiLabelClassificationArgs(num_train_epochs=1)

# Create a MultiLabelClassificationModel
model = MultiLabelClassificationModel(
    "bert",
    "bert-base-chinese",
    num_labels=label_data.len,
    args=model_args,
)

# Train the model
model.train_model(label_data.train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(
    label_data.test
)

